# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f50cf132670>
├── 'a' --> tensor([[-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355]])
└── 'x' --> <FastTreeValue 0x7f50cf1325b0>
    └── 'c' --> tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                        [ 1.4813, -0.5058, -1.3405, -1.7110],
                        [ 0.9798, -0.3347, -0.7590, -0.9877]])

In [4]:
t.a

tensor([[-1.0238,  1.3775, -1.0908],
        [-1.3892,  2.8328,  0.4355]])

In [5]:
%timeit t.a

61.5 ns ± 1.76 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f50cf132670>
├── 'a' --> tensor([[-0.4584, -0.5655, -1.9477],
│                   [ 1.1521, -0.5723, -0.7696]])
└── 'x' --> <FastTreeValue 0x7f50cf1325b0>
    └── 'c' --> tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                        [ 1.4813, -0.5058, -1.3405, -1.7110],
                        [ 0.9798, -0.3347, -0.7590, -0.9877]])

In [7]:
%timeit t.a = new_value

71.4 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0238,  1.3775, -1.0908],
               [-1.3892,  2.8328,  0.4355]]),
    x: Batch(
           c: tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877]]),
       ),
)

In [10]:
b.a

tensor([[-1.0238,  1.3775, -1.0908],
        [-1.3892,  2.8328,  0.4355]])

In [11]:
%timeit b.a

55.3 ns ± 0.561 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0773,  0.3708, -1.5048],
               [-1.6002,  0.8872,  0.8143]]),
    x: Batch(
           c: tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877]]),
       ),
)

In [13]:
%timeit b.a = new_value

475 ns ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

773 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

9.45 µs ± 72.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

135 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f50cc0994f0>
├── 'a' --> tensor([[[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]],
│           
│                   [[-1.0238,  1.3775, -1.0908],
│                    [-1.3892,  2.8328,  0.4355]]])
└── 'x' --> <FastTreeValue 0x7f5015359f70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

30.1 µs ± 703 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f50cc0c5280>
├── 'a' --> tensor([[-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355],
│                   [-1.0238,  1.3775, -1.0908],
│                   [-1.3892,  2.8328,  0.4355]])
└── 'x' --> <FastTreeValue 0x7f5026be5250>
    └── 'c' --> tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                        [ 1.4813, -0.5058, -1.3405, -1.7110],
                 

In [23]:
%timeit t_cat(trees)

27.5 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

51 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2772,  2.0066, -0.2151, -0.3569],
                       [ 1.4813, -0.5058, -1.3405, -1.7110],
                       [ 0.9798, -0.3347, -0.7590, -0.9877]],
              
                      [[-1.2772,  2.0066, -0.2151, -0.3569],
                       [ 1.4813, -0.5058, -1.3405, -1.7110],
                       [ 0.9798, -0.3347, -0.7590, -0.9877]],
              
                      [[-1.2772,  2.0066, -0.2151, -0.3569],
                       [ 1.4813, -0.5058, -1.3405, -1.7110],
                       [ 0.9798, -0.3347, -0.7590, -0.9877]],
              
                      [[-1.2772,  2.0066, -0.2151, -0.3569],
                       [ 1.4813, -0.5058, -1.3405, -1.7110],
                       [ 0.9798, -0.3347, -0.7590, -0.9877]],
              
                      [[-1.2772,  2.0066, -0.2151, -0.3569],
                       [ 1.4813, -0.5058, -1.3405, -1.7110],
                       [ 0.9798, -0.3347, -0.7590, -0.9877]],

In [26]:
%timeit Batch.stack(batches)

71 µs ± 931 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877],
                      [-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877],
                      [-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877],
                      [-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877],
                      [-1.2772,  2.0066, -0.2151, -0.3569],
                      [ 1.4813, -0.5058, -1.3405, -1.7110],
                      [ 0.9798, -0.3347, -0.7590, -0.9877],
                      [-1.2772,  2.0066, -0.2151, -0.3569],
                   

In [28]:
%timeit Batch.cat(batches)

153 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 7.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
